In [ ]:
import cloudscraper
import requests
from bs4 import BeautifulSoup
import pandas as pd
from utils import get_stars, get_href
from scraper import scrap_page

In [ ]:
session = requests.Session()
scraper = cloudscraper.create_scraper(
    sess=session,
    browser={
        "browser": "chrome",
        "platform": "windows",
    },
)


In [ ]:
products_list = []
for i in range(1, 51):
    print(i)
    url = f"https://books.toscrape.com/catalogue/page-{i}.html"
    products = scrap_page(url, scraper)
    products_list.append(products)
products_list

In [ ]:
response = scraper.get("https://books.toscrape.com/catalogue/page-1.html")
soup = BeautifulSoup(response.text, "html.parser")

In [ ]:
products = soup.find('ol').find_all('li')

In [ ]:
products[5]

In [ ]:

products_list = []
for product in products:
    idx = product.find(class_='image_container').a.get('href').split('/')[0].split('_')[1]
    image = product.find('img').get('src').split('/')[-1]
    stars = get_stars([r for r in product.find(class_='star-rating').get("class") if r != 'star-rating'][0])
    title = product.find(class_='image_container').a.get('href').split('/')[1].split('_')[0].replace('-', ' ')
    href = product.find(class_='image_container').a.get('href').split('/')[1]
    price = float(product.find(class_="price_color").text[2:])
    in_stock = 0
    if [1 for r in product.find(class_="instock availability").i.get('class') if r == 'icon-ok'][0] == 1 : in_stock = 1
    data = {
        "id": idx,
        "image": image,
        "stars": stars, 
        "title": title, 
        "price": price, 
        "in_stock": in_stock,
    }
    products_list.append(data)
products_list

In [ ]:
columns = ["id", "image", "stars", "title", "price", "in_stock"]
products_data = pd.DataFrame(products_list, columns=columns).set_index('id').reset_index()
products_data

In [ ]:
href = get_href(products_data.iloc[1])
href